In [1]:
from siphon.catalog import TDSCatalog
from xarray.backends import NetCDF4DataStore
import xarray as xr
import rioxarray as rxr

from datetime import datetime, date, timedelta
import os

In [2]:
os.environ['WEST'] = '90.0'
os.environ['EAST'] = '150.0'
os.environ['SOUTH'] = '-15.0'
os.environ['NORTH'] = '15.0'

In [3]:
def get_nitrogen_dioxide(obs_time):
    # Grab THREDDS Data Server Catalog
    URL = 'http://silam.fmi.fi/thredds/catalog/silam_glob_v5_7_1/catalog.xml'
    catalog = TDSCatalog(URL)
    
    # NetCDF Subset Service
    ncss = catalog.datasets[0].subset()
    
    # Query boundary box and variable
    variable = 'cnc_NO2_gas'
    west = float(os.environ.get('WEST'))
    east = float(os.environ.get('EAST'))
    south = float(os.environ.get('SOUTH'))
    north = float(os.environ.get('NORTH'))
    
    # Create subset query
    query = ncss.query()
    query.variables(variable).time(obs_time)
    query.lonlat_box(west=west, east=east, south=south, north=north)
    data = ncss.get_data(query)

    # Load to xarray Dataset
    return xr.open_dataset(NetCDF4DataStore(data)).isel(time = 0)

In [4]:
# Observation time
obs = date.today()
obs_time = datetime(obs.year, obs.month, obs.day, hour = 6)

ds = get_nitrogen_dioxide(obs_time).isel(hybrid = 0)

In [5]:
# Use date string as file_name
date_string = obs.strftime('%Y%m%d')
fname = f'no2_{date_string}.tif'

# Export nitrogen dioxide as COG
ds.cnc_NO2_gas.rio.to_raster(fname)